In [8]:
import sys, os
import warnings
warnings.simplefilter("ignore")

import torch
import json

import numpy as np
import pandas as pd
from matplotlib import pylab as plt

import sklearn
sklearn.set_config(enable_metadata_routing=True)

from mapie.metrics import regression_coverage_score, regression_mean_width_score
from mapie.subsample import BlockBootstrap
from mapie.regression import MapieTimeSeriesRegressor, MapieRegressor

import pickle
from sklearn.metrics import mean_squared_error


In [2]:
model_data = {} 
folder_path = "./results/"

for filename in os.listdir(folder_path):
    if filename.endswith('.pkl'):
        file_path = os.path.join(folder_path, filename)
        with open(file_path, 'rb') as file:
            data = pickle.load(file)
            model_data[filename] = data

In [3]:
start_ind = 2551
end_ind = 2647

# Start and end time
start = '2023-06-28 00:00:00'
end = '2023-06-28 23:45:00'

# Frequency of the time stamps (e.g., 'H' for hourly)
freq = '15T'

# Creating the time stamp sequence
x = pd.date_range(start=start, end=end, freq=freq)
y_true = model_data['trans_dict.pkl']['y_true']
test_range = [i for i in range(start_ind, end_ind)]

In [4]:
model_data['mlp_dict.pkl'].keys()

dict_keys(['y_pred', 'y_pis', 'coverage', 'width', 'y_true'])

In [ ]:
plt.figure(figsize=(12, 6), dpi=100)

plt.plot(x, y_true[start_ind:end_ind], lw=2, label="Test data", c="C1")
for data, name in zip(model_data.values(), ["MLP", "Transformer", "XGB"]):
    plt.plot(x, data['y_pred'][start_ind:end_ind], lw=2, c="C2", label=name)
    plt.fill_between(
        x,
        data['y_pis'][start_ind:end_ind][:, 0, 0],
        data['y_pis'][start_ind:end_ind][:, 1, 0],
        color="C2",
        alpha=0.2,
        label="Prediction intervals",
    )

plt.ylabel("Imbalance (MWh)")
plt.title("MAPIE conformal intervals")
plt.legend()
plt.show()


In [39]:
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots

# Assuming x, y_true, and model_data are already defined
# start_ind = ...  # define start index
# end_ind = ...    # define end index
model_names = ["MLP", "Transformer", "XGB"]
colors = ["red", "green", "blue"]  # Different colors for each model

# Create subplots with 3 rows and 1 column
fig = make_subplots(rows=3, cols=1, subplot_titles=model_names, vertical_spacing=0.1)
# fig = make_subplots(
#     rows=3, 
#     cols=1, 
#     subplot_titles=model_names, 
#     vertical_spacing=0.1,
#     subplot_titles=dict(font=dict(size=20))  # Increase the size of subplot titles
# )

# Add 'Test data' trace only once
y_axis_range = [-650, 650]  # Example range, adjust as needed
for i in range(1, 4):
    fig.update_yaxes(range=y_axis_range, row=i, col=1)
    fig.add_trace(
        go.Scatter(x=x, y=y_true[start_ind:end_ind], mode='lines', 
                   name="Test data", line=dict(color="black"), 
                   showlegend=False if i > 1 else True),
        row=i, col=1
    )
    fig.layout.annotations[i-1].update(font=dict(size=28))

# Adding traces to each subplot
for i, (data, name) in enumerate(zip(model_data.values(), model_names)):
    # Plot model predictions
    fig.add_trace(
        go.Scatter(x=x, y=data['y_pred'][start_ind:end_ind], mode='lines', name=name, line=dict(color=colors[i])),
        row=i+1, col=1
    )
    
    # Plot prediction intervals (not in the legend)
    fig.add_trace(
        go.Scatter(
            x=x, 
            y=data['y_pis'][start_ind:end_ind][:, 0, 0],
            mode='lines',
            line=dict(color=colors[i], width=0),
            showlegend=False
        ),
        row=i+1, col=1
    )
    fig.add_trace(
        go.Scatter(
            x=x, 
            y=data['y_pis'][start_ind:end_ind][:, 1, 0],
            fill='tonexty',
            mode='lines',
            line=dict(color=colors[i], width=0),
            showlegend=False
        ),
        row=i+1, col=1
    )


################### ---------------------------

fig.update_layout(
    font=dict(size=32, family="Times New Roman"), 
    title=dict(x=0.5, y=0.99, text='MAPIE conformal intervals', font=dict(size=50)),
    xaxis_title='Time (h)', 
    legend=dict(orientation="h", x=0.5, xanchor="center", y=-0.06),
    margin=dict(l=20, r=20, t=100, b=20),   # Adjust bottom margin to ensure legend is visible
    plot_bgcolor='white',
    yaxis_title=' ',
    width=1983,  # Width as calculated for 4/5 of A4 paper width
    height=1200,  # Example height, adjust as needed
)
################### ---------------------------
fig.update_xaxes(showline=True, linewidth=1, linecolor='black', mirror=True, tickfont=dict(size=32))
fig.update_yaxes(showline=True, linewidth=1, linecolor='black', mirror=True, tickfont=dict(size=32))

# Removing x-axis labels and center y-axis title
for i in range(1, 4):
    fig.update_xaxes(title_text="", row=i, col=1)
    fig.update_yaxes(title_standoff=25, row=i, col=1)

# Custom centralized y-axis label using annotation
fig.add_annotation(
    text="System imbalance (MW)",  # Y-axis label
    xref="paper", yref="paper",
    x=-0.075, y=0.5,  # Adjust x for horizontal position, y=0.5 centers it vertically
    showarrow=False,
    textangle=-90,  # Vertical text
    font=dict(size=32)
)

# Show plot
fig.show(config={'staticPlot': True})
pio.write_image(fig, 'mapie_res.pdf')